# Train Music Transformer
Since Fri. May 13th, 2022

Set up training in colab


## Setup


### Ipython



In [1]:
%load_ext autoreload
%autoreload 2



### Colab



In [ ]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = ""


import os
import sys

import torch


if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    ! pip3 install sty icecream music21 transformers datasets
    ! pip3 install stefutils

    path = os.path.join('drive', 'My Drive', 'Research', 'Music with NLP', 'Symbolic-Music-Generation')
    sys.path.append(path)
    ! ls "{path}"


    import time, os
    os.environ['TZ'] = 'US/Eastern'
    time.tzset()

    if torch.cuda.is_available():
        %env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128

if torch.cuda.is_available():
    ! nvidia-smi


from icecream import ic

from musicnlp.util import *

ic(u.proj_dir, u.pkg_nm)



### code

In [ ]:
import transformers

from musicnlp.trainer import train

transformers.set_seed(sconfig('random-seed'))



## Prep for training


In [ ]:
md_nm = 'transf-xl'
# md_sz = 'debug'
# md_sz = 'debug-large'
md_sz = 'base'
# n = 32
# n = None
n = 1024
ic(md_sz, n)

augment_key = False

max_length = 512
# mem_len = 256
mem_len = 128
model_config = dict(max_length=max_length)   # TODO: short length to test for now
num_train_epochs = 4  # TODO

pop = 'musicnlp music extraction, dnm=POP909, n=909, meta={mode=melody, prec=5, th=1}, 2022-05-20_14-52-04'
mst = 'musicnlp music extraction, dnm=MAESTRO, n=1276, meta={mode=melody, prec=5, th=1}, 2022-05-20_14-52-28'
lmd = 'musicnlp music extraction, dnm=LMD, n=176640, meta={mode=melody, prec=5, th=1}, 2022-05-27_15-23-20'
dnms = [pop, mst, lmd]
my_train_args = dict(
    augment_key=augment_key,
    save_epochs=2
)
train_args = dict()

with_tqdm = True
if with_tqdm:
    my_train_args.update(dict(tqdm=True, logging_strategy='epoch'))
if 'debug' not in md_sz:
    model_config.update(mem_len=mem_len)  # TODO
    train_args = dict(
        gradient_checkpointing=True,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=num_train_epochs
    )
mdl, tokenizer, trainer = train.get_all_setup(
    model_name=md_nm, model_size=md_sz, dataset_names=dnms, model_config=model_config,
    n_sample=n, train_args=train_args, my_train_args=my_train_args
)



## Train


Check log and tensorboard files written



In [5]:
ignore_keys_for_eval = ['losses', 'mems', 'hidden_states', 'attentions']
trainer.train(ignore_keys_for_eval=ignore_keys_for_eval)
trainer.save_model(os.path.join(trainer.args.output_dir, 'trained'))



ValueError: ignored

## Sanity check



In [ ]:
ic(trainer.args.output_dir)
ic(os.listdir(trainer.args.output_dir))

